<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/Datenanalyse/EDA_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
## import modules

import io, os, sys, setuptools, tokenize
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from datetime import datetime
from datetime import timedelta
from math import sqrt

In [13]:
# Größe der Plots festlegen
matplotlib.rcParams['figure.figsize'] = (16, 9)

In [14]:
# Allgemeine Angaben
latitude = 50.2
longitude = 7.8

seconds_in_day = 24*60*60
seconds_in_year = (365.2425)*seconds_in_day

# tz, altitude, name = 'Europe/Berlin', 70, 'SanktAugustin'
# tus = pvlib.location.Location(latitude, longitude, tz, altitude, name)

In [15]:
# Pfad zur CSV-Datei auf Google Drive
name_Messwerte = 'Messdaten_CAMS_GHI.csv'
name_Vorhersage = 'Vorhersagedaten_CAMS_VarIdx.csv'

folder_import = '/content/drive/My Drive/Colab_Notebooks/Clean_Data/'
folder_export = '/content/drive/My Drive/Colab_Notebooks/plot_daily_UVI/'

In [16]:
df_Vorhersage = pd.read_csv(folder_import + name_Vorhersage)
df_Vorhersage = pd.to_datetime(df_Vorhersage['Datetime'])

# df_Vorhersage.set_index('Datetime', inplace=True)
# df_Vorhersage.index = pd.to_datetime(df_Vorhersage.index)

In [17]:
df_Messwerte = pd.read_csv(folder_import + name_Messwerte)
df_Messwerte['Datetime'] = pd.to_datetime(df_Messwerte['Datetime'])

# df_Messwerte.set_index('Datetime', inplace=True)
# df_Messwerte.index = pd.to_datetime(df_Messwerte.index)

In [18]:
df_Vorhersage

,Datetime
0,2022-06-15 07:00:00
1,2022-06-15 08:00:00
2,2022-06-15 09:00:00
3,2022-06-15 10:00:00
4,2022-06-15 11:00:00
...,...
2865,2023-07-25 14:00:00
2866,2023-07-25 15:00:00
2867,2023-07-25 16:00:00
2868,2023-07-25 17:00:00


## Tägliche Durchschnittswerte der gemessenen UV und Glo-Strahlung

In [19]:
# Display basic information about the datasets
info_Vorhersage = df_Vorhersage.info()
info_Messwerte = df_Messwerte.info()

print(info_Messwerte)

<class 'pandas.core.series.Series'>
RangeIndex: 2870 entries, 0 to 2869
Series name: Datetime
Non-Null Count  Dtype         
--------------  -----         
2870 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 22.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96732 entries, 0 to 96731
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Datetime            96732 non-null  datetime64[ns]
 1   Observation_period  96732 non-null  object        
 2   Clear_sky_GHI       96732 non-null  float64       
 3   Clear_sky_BHI       96732 non-null  float64       
 4   GHI                 96732 non-null  float64       
 5   BHI                 96732 non-null  float64       
 6   UVI                 96732 non-null  float64       
 7   UVA                 96732 non-null  float64       
 8   UVB                 96732 non-null  float64       
 9   erythem             96732 non-null 

In [20]:
# Vorhersage und Messung zusammenfügen:

df_merged = pd.merge(df_Messwerte, df_Vorhersage, on = 'Datetime', how = 'inner')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811 entries, 0 to 2810
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Datetime            2811 non-null   datetime64[ns]
 1   Observation_period  2811 non-null   object        
 2   Clear_sky_GHI       2811 non-null   float64       
 3   Clear_sky_BHI       2811 non-null   float64       
 4   GHI                 2811 non-null   float64       
 5   BHI                 2811 non-null   float64       
 6   UVI                 2811 non-null   float64       
 7   UVA                 2811 non-null   float64       
 8   UVB                 2811 non-null   float64       
 9   erythem             2811 non-null   float64       
 10  Datum               2811 non-null   object        
 11  Uhrzeit             2811 non-null   object        
 12  Messzeitpunkt       2811 non-null   float64       
 13  ghi                 2811 non-null   float64     

## Tägliche/Stündliche Durchschnittswerte der gemessenen UV und Glo-Strahlung

In [21]:
# Calculating MAE and RMSE for uvbed predictions vs. UVI measurements
mae_uv = np.mean(np.abs(df_merged['UVI'] - df_merged['uvbed']))
rmse_uv = np.sqrt(np.mean((df_merged['UVI'] - df_merged['uvbed']) ** 2))

# Calculating MAE and RMSE for uvbed predictions vs. UVI measurements
mae_uvcs = np.mean(np.abs(df_merged['UVI'] - df_merged['uvbedcs']))
rmse_uvcs = np.sqrt(np.mean((df_merged['UVI'] - df_merged['uvbedcs']) ** 2))

KeyError: 'uvbed'

In [ ]:
  print(mae_uv, rmse_uv, mae_uvcs, rmse_uvcs)

In [ ]:
# Bestimme des Tages und der Stunde für jede Zeile
df_merged['Date'] =df_merged.index.date
df_merged['Hour'] = df_merged.index.hour

# Bestimmen der Täglichen und stündlichen Mittelwerte
daily_aggregates = df_merged.groupby('Date')[['GHI', 'UVI', 'uvbed', 'uvbedcs']].mean()
hourly_aggregates = df_merged.groupby('Hour')[['GHI', 'UVI', 'uvbed', 'uvbedcs']].mean()

daily_aggregates.index = pd.to_datetime(daily_aggregates.index)
weekly_aggregates = daily_aggregates.resample('W').mean()

In [ ]:
# 1. Wöchentlicher Durchschnitt
plt.figure(figsize=(10, 6))
plt.plot(weekly_aggregates.index, weekly_aggregates['UVI'], label='UVI (Messung)')
plt.plot(weekly_aggregates.index, weekly_aggregates['uvbed'], label='UVI (Vorhersage)', linestyle='--')
plt.plot(weekly_aggregates.index, weekly_aggregates['uvbedcs'], label='UVI Clear Sky (Vorhersage)', linestyle='-.')
plt.title('Wöchentlicher Durchschnitt des UVI')
plt.xlabel('Monat')
plt.ylabel('UV-Index')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# 2. Stündlicher Durchschnitt
plt.figure(figsize=(10, 6))
plt.plot(hourly_aggregates.index, hourly_aggregates['UVI'], label='UVI (Messung)')
plt.plot(hourly_aggregates.index, hourly_aggregates['uvbed'], label='UVI (Vorhersage)', linestyle='--')
plt.plot(hourly_aggregates.index, hourly_aggregates['uvbedcs'], label='UVI Clear Sky (Vorhersage)', linestyle='-.')
plt.title('Stündlicher Durchschnitt des UVI')
plt.xlabel('Tageszeit')
plt.ylabel('UV-Index')
plt.legend()
plt.grid()
plt.show()

## Anomaly Detektion:

In [ ]:
# Berechnen der residuen
df_merged['Residuals_UV'] = df_merged['UVI'] - df_merged['uvbed']
df_merged['Residuals_UVcs'] = df_merged['UVI'] - df_merged['uvbedcs']

Erste Analyse:

1. Zeitliche Abdeckung
2. Vorhersagedaten
3. Gemeinsamer Zeitraum

Vergleich zwischen gemessenen und vorhergesagten Daten:

MAE:

RSME:

## Allgemeine Beobachtungen zu den UVI-Messwerten

## Anomalierkennung

Anaomalien wurden auf Basis von 2 Residuen (> 2 Standartabweichungen) identifiziert.

## Saisonale Analyse

## Bewölkungseffekt

## Fehleranalyse nach Stunden und Tagen

## Residuals über die Zeit

## Täglicher Durchschitt der UVI Werte

## Monatlicher Durchschitt der UVI Werte

## Durchschnittliche Residuals in Abhängigkeit der Cloud Cover Levels

## Montalicher Durchschnitt der Residuen

## Größte positive und negative Residuen

## Monatliche Residuen Trends

## Sonstiges bzw. alt...

In [ ]:
# 2. Stündliche Durchschnittswerte
figsize=(10, 6)
title = 'Vergleich der stündlichen Durchschnittswerte'
x_label = 'Tageszeit'
y1_color = 'b'
y1_label = 'UVI'
y2_color = 'r'
y2_label = 'uvbedcs'
y3_color = 'g'
y3_label = 'uvbed'


fig, ax1 = plt.subplots(figsize = figsize)
ax1.plot(hourly_aggregates.index, hourly_aggregates['UVI'], color = y1_color, label = y1_label)
ax1.plot(hourly_aggregates.index, hourly_aggregates['uvbed'], color = y3_color, label = y3_label)
ax1.set_xlabel(x_label)
ax1.set_ylabel(y1_label, color = y1_color)
ax1.tick_params(axis='y', labelcolor=y1_color)

ax2 = ax1.twinx()
# ax2.plot(hourly_aggregates.index, hourly_aggregates['uvbedcs'], color = y2_color, label = y2_label)
ax2.set_ylabel(y2_label, color = y2_color)
ax2.tick_params(axis='y', labelcolor=y2_color)

# Titel und Legenden
fig.suptitle(title)
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")

# Plot anzeigen
plt.show()